# <b> Database </b>

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. DynamoDB

In [2]:
import time
import boto3
from utils.ddb import ddb_constructor
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")
ddb_construction = ddb_constructor(region_name=pm.get_params(key=strPrefix+"REGION"))

### 1.1 Create table: Inference-results (모델 배치추론결과)

In [4]:
strDDBTableName = strPrefix+'inference-results'

In [5]:
ddb_construction.create_table(
    AttributeDefinitions=[
        {'AttributeName': 'store_id', 'AttributeType': 'N'}, # 'S'|'N'|'B' 
    ],
    TableName=strDDBTableName,
    KeySchema=[
        {'AttributeName': 'store_id', 'KeyType': 'HASH'}, # 'HASH'(partition)|'RANGE'(sort)
    ],
    BillingMode='PAY_PER_REQUEST',# 'PROVISIONED'|'PAY_PER_REQUEST'('ProvisionedThroughput' requried),  
    #ProvisionedThroughput={'ReadCapacityUnits': 123, 'WriteCapacityUnits': 123},
)
pm.put_params(key=strPrefix+"INFERENCE-RES-TABLE-NAME", value=strDDBTableName, overwrite=True)

dynamodb.Table(name='API-GATEWAY-LAMBDA-DEMO-inference-results') was created!!


'Store suceess'

### 1.2 Insert records

In [11]:
import json
from decimal import Decimal
from utils.ddb import ddb_handler

In [13]:
ddb_inf_res = ddb_handler(strTableName=strDDBTableName, region_name=pm.get_params(key=strPrefix+"REGION"))
time.sleep(2)

Table [API-GATEWAY-LAMBDA-DEMO-inference-results] is ACTIVE now.


In [14]:
dicRecord = {
    "store_id": 106561002,
    "info": [
        {"유사판매처ID": 169081000, "적용경로": "TA", "생성여부": 0, "유사도": 0.47},
        {"유사판매처ID": 238671000, "적용경로": "TA", "생성여부": 0, "유사도": 0.46},
        {"유사판매처ID": 241381000, "적용경로": "TA", "생성여부": 0, "유사도": 0.45},
        {"유사판매처ID": 243501000, "적용경로": "TA", "생성여부": 0, "유사도": 0.44},
        {"유사판매처ID": 244231000, "적용경로": "TA", "생성여부": 0, "유사도": 0.43},
        {"유사판매처ID": 248941000, "적용경로": "TA", "생성여부": 0, "유사도": 0.42},
        {"유사판매처ID": 252621000, "적용경로": "TA", "생성여부": 0, "유사도": 0.41},
        {"유사판매처ID": 252771001, "적용경로": "TA", "생성여부": 0, "유사도": 0.40},
        {"유사판매처ID": 254721000, "적용경로": "TA", "생성여부": 0, "유사도": 0.39},
        {"유사판매처ID": 277151000, "적용경로": "TA", "생성여부": 0, "유사도": 0.38},
        {"유사판매처ID": 282381000, "적용경로": "TA", "생성여부": 0, "유사도": 0.37},
        {"유사판매처ID": 511551008, "적용경로": "TA", "생성여부": 0, "유사도": 0.36},
        {"유사판매처ID": 522401009, "적용경로": "TA", "생성여부": 0, "유사도": 0.35},
        {"유사판매처ID": 522401010, "적용경로": "TA", "생성여부": 0, "유사도": 0.34},
        {"유사판매처ID": 523361003, "적용경로": "TA", "생성여부": 0, "유사도": 0.33},
        {"유사판매처ID": 523361004, "적용경로": "TA", "생성여부": 0, "유사도": 0.32},
        {"유사판매처ID": 523701001, "적용경로": "TA", "생성여부": 0, "유사도": 0.31},
        {"유사판매처ID": 533911000, "적용경로": "TA", "생성여부": 0, "유사도": 0.30},
        {"유사판매처ID": 542751001, "적용경로": "TA", "생성여부": 0, "유사도": 0.29},
        {"유사판매처ID": 550111000, "적용경로": "TA", "생성여부": 0, "유사도": 0.28},
    ]
}

In [15]:
dicItem = json.loads(json.dumps(dicRecord), parse_float=Decimal)
ddb_inf_res.put_item(dicItem)
time.sleep(2)

### 1.3 Check records

In [16]:
from operator import itemgetter

In [17]:
dicKey = {"store_id":106561002}
dicRes = ddb_inf_res.get_item(dicKey)

nStoreID = int(dicRes["store_id"])
listInfo = dicRes["info"]
listInfo = sorted(listInfo, key=itemgetter('유사도'), reverse=True)
listSimStoreIDs = [int(dicInfo["유사판매처ID"]) for dicInfo in listInfo]
listSimStoreIDs


[169081000,
 238671000,
 241381000,
 243501000,
 244231000,
 248941000,
 252621000,
 252771001,
 254721000,
 277151000,
 282381000,
 511551008,
 522401009,
 522401010,
 523361003,
 523361004,
 523701001,
 533911000,
 542751001,
 550111000]

### 1.4 Truncate table (optional)

In [12]:
ddb_inf_res.truncate_table()

Deleted 1
